In [1]:
from qiskit import QuantumProgram
from IBMQuantumExperience import IBMQuantumExperience
import Qconfig
api = IBMQuantumExperience(Qconfig.APItoken, {'url':Qconfig.config["url"]})
from qiskit.backends import discover_remote_backends
remote_backends = discover_remote_backends(api)

The following defines and runs an implementation of the $d=7$ repetition code with a stored `0`.

In [2]:
num = 16
device = 'ibmqx5'
shots = 8192

In [3]:
Q_program = QuantumProgram()
# declare register of qubits
q = Q_program.create_quantum_register("q", num)
# declare register of classical bits to hold measurement results
c = Q_program.create_classical_register("c", num)
# create circuit
repetitionScript = Q_program.create_circuit("repetitionScript", [q], [c])  

repetitionScript.barrier()

repetitionScript.cx(q[15],q[14])
repetitionScript.cx(q[11],q[10])
#repetitionScript.cx(q[9],q[8])
repetitionScript.cx(q[5],q[4])
repetitionScript.cx(q[1],q[0])
repetitionScript.h(q[13])
repetitionScript.h(q[12])
repetitionScript.cx(q[12],q[13])
repetitionScript.h(q[12])
repetitionScript.h(q[13])
repetitionScript.h(q[3])
repetitionScript.h(q[2])
repetitionScript.cx(q[2],q[3])
repetitionScript.h(q[2])
repetitionScript.h(q[3])

repetitionScript.barrier()

repetitionScript.cx(q[13],q[14])
repetitionScript.cx(q[9],q[10])
repetitionScript.cx(q[3],q[4])
repetitionScript.cx(q[1],q[2])
repetitionScript.cx(q[15],q[0])
repetitionScript.h(q[12])
repetitionScript.h(q[11])
repetitionScript.cx(q[12],q[11])
repetitionScript.h(q[11])
repetitionScript.h(q[12])
#repetitionScript.h(q[8])
#repetitionScript.h(q[7])
#repetitionScript.cx(q[8],q[7])
#repetitionScript.h(q[7])
#repetitionScript.h(q[8])

repetitionScript.barrier()

for address in range(num):
    repetitionScript.measure(q[address], c[address])

Q_program.execute(["repetitionScript"], backend=device, shots = shots, max_credits = 5, timeout=1)  

We'll now get the results, assuming that this was the last job to run. The full info will be printed to screen as a record, but the actual results will also go on to be processed in the cells after.

In [5]:
full_info = api.get_jobs(limit=5)[0]

for key in ['id','creationDate']:
    print( "\n*"+str(key)+"*\n\n"+full_info[key] )

for key in full_info['qasms'][0]:
    print("\n*"+key+"*\n\n", full_info['qasms'][0][key] )
    

results = full_info['qasms'][0]['result']['data']['counts']


*id*

002c53c801d7c3fe9e42c11f130292a4

*creationDate*

2018-04-30T22:07:50.564Z

*qasm*

 
include "qelib1.inc";
qreg q[16];
creg c[16];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15];
cx q[15],q[14];
u2(0,3.14159265358979) q[13];
u2(0,3.14159265358979) q[12];
cx q[11],q[10];
cx q[5],q[4];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[2];
cx q[1],q[0];
cx q[12],q[13];
cx q[2],q[3];
u2(0,3.14159265358979) q[13];
u2(0,3.14159265358979) q[12];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[2];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15];
cx q[13],q[14];
u2(0,3.14159265358979) q[12];
u2(0,3.14159265358979) q[11];
cx q[9],q[10];
cx q[3],q[4];
cx q[1],q[2];
cx q[15],q[0];
cx q[12],q[11];
u2(0,3.14159265358979) q[12];
u2(0,3.14159265358979) q[11];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15];
measure q[15] -> c[15];
measure q[14] ->

The probability that each qubit is output with value `1` is determined and printed. The [standard qubit numbering](https://github.com/QISKit/ibmqx-backend-information/blob/master/backends/ibmqx5/README.md) is used. To flag up the higher probabilities, they are printed further toward the right.

In [6]:
p = [0]*num
for qubit in range(num):
    for string in results.keys():
        if string[num-1-qubit]=='1':
            p[qubit] += results[string]/shots
            
    print(qubit, " "*int(15*p[qubit]) + str(p[qubit]) )

0   0.18994140625
1   0.140625
2    0.2196044921875
3   0.134765625
4   0.1563720703125
5 0.0313720703125
6 0.016357421875
7 0.0113525390625
8 0.025390625
9  0.0821533203125
10  0.10546875
11 0.0546875
12   0.14111328125
13 0.063720703125
14   0.1439208984375
15   0.1583251953125
